# Monte Carlo simulation of SIS epidemic spreading


### Import libraries

In [1]:
import igraph as ig
import matplotlib.pyplot as plt
import numpy as np
import time


%load_ext cython

### Simulation function
Performs 1 full simulation

In [2]:
%%cython --annotate
import numpy as np
import igraph as ig
from libc.math cimport pow
from libc.stdlib cimport rand, srand, RAND_MAX
from libc.time cimport time,time_t

def simulation(g, int len_g, char[:] state, double mu, double beta):
        
    # Constants
    cdef double P0 = 0.2
    cdef int TMAX = 1000
    cdef int TTRANS = 900
    cdef double[:] infected_ratio_list = np.zeros(TMAX - TTRANS)
    cdef long start_time
    cdef int i, j, v, n, infected_vertices
    cdef double _beta = 1 - beta
    cdef char[:] old_state = np.zeros(len_g, dtype = np.int8)
    cdef double prob
    cdef double a = RAND_MAX
    
    # set random seed 
    srand(time(NULL)) 
    #srand(time.time())
    
    #simulation loop
    for i in range(TMAX):
        
        # copy old state
        for j in range(len_g):
            old_state[j] = state[j]
        #old_state = np.copy(state)
        
        # update states of all nodes
        for j in range(len_g):
            
            # if old node state is I
            if old_state[j]:
                prob = rand() / a # generate random number in [0, 1]
                if prob < mu: # update new state
                    state[j] = 0 
                else:
                    state[j] = 1
                
            # if old node state is S
            else:
                infected_vertices = 0  # counter of the infected vertices of current SUCSECTIBLE node
                
                for n in g.neighbors(j):
                    infected_vertices += old_state[n]
                #infected_vertices = sum([old_state[v] for v in g.neighbors(j)]) # number of infected neighbors
                prob = rand() / a
                if prob < pow(1 - beta, infected_vertices):
                    state[j] = 0
                else:
                    state[j] = 1
                    
            # we compute infection ratio from iteration 900
            if i >= TTRANS:
                infected_ratio_list[i - TTRANS] += state[j]
                infected_ratio_list[i - TTRANS] /= len_g

        
    return np.array(infected_ratio_list)

In [3]:
MU = 0.5
BETA = 0.5
P0 = 0.2

### Create network

In [4]:
len_g = 1000
g = ig.Graph.Barabasi(len_g, 5)

### Perform simulation

In [5]:
state = np.array(np.random.binomial(1, P0, size=len_g), dtype=np.int8) # initial state
#old_state = np.copy(state) # initial old state

start_time = time.time()
for i in range(100): # perform 30 simulations
    start = time.time()
    a = simulation(g, len_g, state, MU, BETA) # performs 1 full simulaiton of 1000 steps
    print("Simulation number {} requires {} seconds".format(i, time.time()-start))
print("Time required for 100 simulations: {}".format(time.time()-start_time))

Simulation number 0 requires 0.23096203804016113 seconds
Simulation number 1 requires 0.24420762062072754 seconds
Simulation number 2 requires 0.23922276496887207 seconds
Simulation number 3 requires 0.23121976852416992 seconds
Simulation number 4 requires 0.24016618728637695 seconds
Simulation number 5 requires 0.2689402103424072 seconds
Simulation number 6 requires 0.2630927562713623 seconds
Simulation number 7 requires 0.26927781105041504 seconds
Simulation number 8 requires 0.25577545166015625 seconds
Simulation number 9 requires 0.2812514305114746 seconds
Simulation number 10 requires 0.24231648445129395 seconds
Simulation number 11 requires 0.25334954261779785 seconds
Simulation number 12 requires 0.25722408294677734 seconds
Simulation number 13 requires 0.24918389320373535 seconds
Simulation number 14 requires 0.271348237991333 seconds
Simulation number 15 requires 0.27426600456237793 seconds
Simulation number 16 requires 0.28415369987487793 seconds
Simulation number 17 requires